In [ ]:
class Piece:
    """Базовый класс для всех фигур"""
    def __init__(self, color, symbol):
        self.color = color
        self.symbol = symbol

    def __str__(self):
        return self.symbol

    def valid_moves(self, board, position):
        raise NotImplementedError("Subclasses should implement this method")


class Checker(Piece):
    """Класс шашки"""
    def __init__(self, color, symbol, is_king=False):
        super().__init__(color, symbol)
        self.is_king = is_king  # является ли шашка дамкой

    def valid_moves(self, board, position):
        """Возвращает список допустимых ходов для шашки"""
        row, col = position
        moves = []
        capture_moves = []
        direction = -1 if self.color == 'white' else 1

        if not self.is_king:
            for dc in (-1, 1):
                r, c = row + direction, col + dc
                if 0 <= r < 8 and 0 <= c < 8 and not board[r][c]:
                    moves.append((r, c))

            for dc in (-2, 2):
                r, c = row + 2 * direction, col + dc
                mid_r, mid_c = row + direction, col + dc // 2
                if (0 <= r < 8 and 0 <= c < 8 and not board[r][c] and
                    board[mid_r][mid_c] and board[mid_r][mid_c].color != self.color):
                    capture_moves.append((r, c))
        else:
            for dr in (-1, 1):
                for dc in (-1, 1):
                    r, c = row + dr, col + dc
                    while 0 <= r < 8 and 0 <= c < 8:
                        if not board[r][c]:
                            moves.append((r, c))
                        else:
                            if board[r][c].color != self.color and 0 <= r + dr < 8 and 0 <= c + dc < 8 and not board[r + dr][c + dc]:
                                capture_moves.append((r + dr, c + dc))
                            break
                        r += dr
                        c += dc

        return capture_moves if capture_moves else moves


class CheckerKing(Checker):
    """Класс дамки в шашках (наследуется от обычной шашки)"""
    def __init__(self, color, symbol):
        super().__init__(color, symbol, is_king=True)


class Pawn(Piece):
    """Класс пешки"""
    def valid_moves(self, board, position):
        row, col = position
        moves = []
        direction = -1 if self.color == 'white' else 1
        start_row = 6 if self.color == 'white' else 1

        if 0 <= row + direction < 8 and not board[row + direction][col]:
            moves.append((row + direction, col))
            if row == start_row and not board[row + 2 * direction][col]:
                moves.append((row + 2 * direction, col))

        for dc in (-1, 1):
            r, c = row + direction, col + dc
            if 0 <= c < 8 and 0 <= r < 8:
                target = board[r][c]
                if target and target.color != self.color:
                    moves.append((r, c))

        return moves


class Rook(Piece):
    """Класс ладьи"""
    def valid_moves(self, board, position):
        row, col = position
        moves = []
        directions = ((-1, 0), (1, 0), (0, -1), (0, 1))

        for dr, dc in directions:
            r, c = row + dr, col + dc
            while 0 <= r < 8 and 0 <= c < 8:
                target = board[r][c]
                if not target:
                    moves.append((r, c))
                else:
                    if target.color != self.color:
                        moves.append((r, c))
                    break
                r += dr
                c += dc

        return moves


class Knight(Piece):
    """Класс коня"""
    def valid_moves(self, board, position):
        row, col = position
        moves = []
        moves_pattern = ((-2, -1), (-1, -2), (1, -2), (2, -1),
                         (2, 1), (1, 2), (-1, 2), (-2, 1))

        for dr, dc in moves_pattern:
            r, c = row + dr, col + dc
            if 0 <= r < 8 and 0 <= c < 8:
                target = board[r][c]
                if not target or target.color != self.color:
                    moves.append((r, c))

        return moves


class Bishop(Piece):
    """Класс слона"""
    def valid_moves(self, board, position):
        row, col = position
        moves = []
        directions = ((-1, -1), (-1, 1), (1, -1), (1, 1))

        for dr, dc in directions:
            r, c = row + dr, col + dc
            while 0 <= r < 8 and 0 <= c < 8:
                target = board[r][c]
                if not target:
                    moves.append((r, c))
                else:
                    if target.color != self.color:
                        moves.append((r, c))
                    break
                r += dr
                c += dc

        return moves


class Queen(Piece):
    """Класс ферзя"""
    def valid_moves(self, board, position):
        rook = Rook(self.color, 'Q')
        bishop = Bishop(self.color, 'Q')
        return rook.valid_moves(board, position) + bishop.valid_moves(board, position)


class King(Piece):
    """Класс короля"""
    def valid_moves(self, board, position):
        row, col = position
        moves = []

        for dr in (-1, 0, 1):
            for dc in (-1, 0, 1):
                if dr == 0 and dc == 0:
                    continue
                r, c = row + dr, col + dc
                if 0 <= r < 8 and 0 <= c < 8:
                    target = board[r][c]
                    if not target or target.color != self.color:
                        moves.append((r, c))

        return moves


# Новые фигуры:

class Archer(Piece):
    """Лучник - ходит как ладья, но на 3 клетки и может перепрыгивать через фигуры"""
    def valid_moves(self, board, position):
        row, col = position
        moves = []
        directions = ((-1, 0), (1, 0), (0, -1), (0, 1))

        for dr, dc in directions:
            for step in range(1, 4):  # Может ходить на 1-3 клетки
                r, c = row + dr * step, col + dc * step
                if 0 <= r < 8 and 0 <= c < 8:
                    target = board[r][c]
                    if not target:
                        moves.append((r, c))
                    else:
                        if target.color != self.color:
                            moves.append((r, c))
                        break  # Может перепрыгивать, но не может бить через фигуры

        return moves


class Dragon(Piece):
    """Дракон - ходит буквой Г (как конь) + на 1 клетку в любом направлении"""
    def valid_moves(self, board, position):
        row, col = position
        moves = []
        # Ходы как у коня
        knight_moves = ((-2, -1), (-1, -2), (1, -2), (2, -1),
                        (2, 1), (1, 2), (-1, 2), (-2, 1))
        # Ходы на 1 клетку в любом направлении
        king_moves = [ (dr, dc) for dr in (-1, 0, 1) for dc in (-1, 0, 1) if not (dr == 0 and dc == 0) ]

        for dr, dc in knight_moves + king_moves:
            r, c = row + dr, col + dc
            if 0 <= r < 8 and 0 <= c < 8:
                target = board[r][c]
                if not target or target.color != self.color:
                    moves.append((r, c))

        return moves


class Wizard(Piece):
    """Волшебник - ходит на 2 клетки по диагонали или ортогонально, может перепрыгивать фигуры"""
    def valid_moves(self, board, position):
        row, col = position
        moves = []
        directions = ((-2, -2), (-2, 0), (-2, 2),
                      (0, -2),          (0, 2),
                      (2, -2),  (2, 0), (2, 2))

        for dr, dc in directions:
            r, c = row + dr, col + dc
            if 0 <= r < 8 and 0 <= c < 8:
                target = board[r][c]
                if not target or target.color != self.color:
                    moves.append((r, c))

        return moves


class Board:
    """Класс, представляющий шахматную доску и игровую логику"""
    def __init__(self, game_type='chess'):
        self.board = [[None for _ in range(8)] for _ in range(8)]
        self.move_history = []
        self.game_type = game_type
        self.setup_board()

    def setup_board(self):
        """Расставляет фигуры в начальную позицию"""
        if self.game_type == 'chess':
            self.board[0] = [
                Rook('black', 'r'), Knight('black', 'n'), Bishop('black', 'b'), Queen('black', 'q'),
                King('black', 'k'), Archer('black', 'a'), Dragon('black', 'd'), Rook('black', 'r')
            ]
            self.board[1] = [Pawn('black', 'p') for _ in range(8)]

            self.board[6] = [Pawn('white', 'P') for _ in range(8)]
            self.board[7] = [
                Rook('white', 'R'), Dragon('white', 'D'), Archer('white', 'A'), Queen('white', 'Q'),
                King('white', 'K'), Bishop('white', 'B'), Knight('white', 'N'), Rook('white', 'R')
            ]
        elif self.game_type == 'chess_with_wizard':
            # Вариант с волшебником вместо ферзя
            self.board[0] = [
                Rook('black', 'r'), Knight('black', 'n'), Bishop('black', 'b'), Wizard('black', 'w'),
                King('black', 'k'), Bishop('black', 'b'), Knight('black', 'n'), Rook('black', 'r')
            ]
            self.board[1] = [Pawn('black', 'p') for _ in range(8)]

            self.board[6] = [Pawn('white', 'P') for _ in range(8)]
            self.board[7] = [
                Rook('white', 'R'), Knight('white', 'N'), Bishop('white', 'B'), Wizard('white', 'W'),
                King('white', 'K'), Bishop('white', 'B'), Knight('white', 'N'), Rook('white', 'R')
            ]
        elif self.game_type == 'checkers':
            for row in range(8):
                for col in range(8):
                    if (row + col) % 2 == 1:
                        if row < 3:
                            self.board[row][col] = Checker('black', 'b')
                        elif row > 4:
                            self.board[row][col] = Checker('white', 'w')

    def display(self):
        """Отображает текущее состояние доски в терминале"""
        print("  a b c d e f g h")
        for i, row in enumerate(self.board):
            print(f"{8 - i} ", end="")
            print(' '.join(str(piece) if piece else '.' for piece in row), f"{8 - i}")
        print("  a b c d e f g h")

    def move_piece(self, start, end, color):
        start_row, start_col = start
        end_row, end_col = end
        piece = self.board[start_row][start_col]

        if not piece or piece.color != color:
            print("Некорректный ход: вы пытаетесь походить чужой фигурой или пустой клеткой.")
            return False

        valid_moves = piece.valid_moves(self.board, start)
        if (end_row, end_col) in valid_moves:
            self.move_history.append((start, end, self.board[end_row][end_col]))

            self.board[end_row][end_col] = piece
            self.board[start_row][start_col] = None

            if self.game_type == 'checkers':
                if isinstance(piece, Checker) and not piece.is_king:
                    if (piece.color == 'white' and end_row == 0) or (piece.color == 'black' and end_row == 7):
                        self.board[end_row][end_col] = CheckerKing(piece.color, piece.symbol.upper())

                if abs(start_row - end_row) == 2:
                    mid_row = (start_row + end_row) // 2
                    mid_col = (start_col + end_col) // 2
                    self.board[mid_row][mid_col] = None

            return True

        print("Некорректный ход: фигура не может так ходить.")
        return False

    def undo_move(self):
        if not self.move_history:
            print("Нет ходов для отмены.")
            return False

        start, end, captured = self.move_history.pop()
        self.board[start[0]][start[1]] = self.board[end[0]][end[1]]
        self.board[end[0]][end[1]] = captured
        return True

    def show_hints(self, position):
        row, col = position
        piece = self.board[row][col]
        current_player = 'white' if len(self.move_history) % 2 == 0 else 'black'

        if not piece or piece.color != current_player:
            print("Некорректная позиция для подсказки.")
            return

        moves = piece.valid_moves(self.board, (row, col))

        for r in range(8):
            for c in range(8):
                print('*' if (r, c) in moves else self.board[r][c] or '.', end=' ')
            print()

    def get_threatened_pieces(self, color):
        """
        Возвращает информацию о фигурах указанного цвета, которые находятся под боем.
        Также проверяет, находится ли король под шахом.
        """
        threatened = []
        in_check = False
        king_pos = None

        for row in range(8):
            for col in range(8):
                piece = self.board[row][col]
                if piece and piece.color == color and isinstance(piece, King):
                    king_pos = (row, col)
                    break
            if king_pos:
                break

        opponent_color = 'black' if color == 'white' else 'white'
        for row in range(8):
            for col in range(8):
                piece = self.board[row][col]
                if piece and piece.color == opponent_color:
                    moves = piece.valid_moves(self.board, (row, col))

                    for move_row, move_col in moves:
                        target = self.board[move_row][move_col]
                        if target and target.color == color:
                            if (move_row, move_col) not in threatened:
                                threatened.append((move_row, move_col))

                            if isinstance(target, King):
                                in_check = True

        return {
            'threatened': threatened,
            'check': in_check
        }

    def display_threats(self, color):
        """
        Отображает доску с выделением угрожаемых фигур и информацию о шахе.
        """
        threats_info = self.get_threatened_pieces(color)
        threatened = threats_info['threatened']
        in_check = threats_info['check']

        print("  a b c d e f g h")
        for i, row in enumerate(self.board):
            print(f"{8 - i} ", end="")
            for j, piece in enumerate(row):
                if (i, j) in threatened:
                    print('\033[91m' + (str(piece) if piece else '.') + '\033[0m', end=' ')
                else:
                    print(str(piece) if piece else '.', end=' ')
            print(f"{8 - i}")
        print("  a b c d e f g h")

        if in_check:
            print("\033[91mШАХ королю!\033[0m")

        if threatened:
            print("Угрожаемые фигуры (подсвечены красным):")
            for row, col in threatened:
                piece = self.board[row][col]
                col_letter = chr(col + ord('a'))
                row_number = 8 - row
                print(f"- {piece} на {col_letter}{row_number}")
        else:
            print("Нет угрожаемых фигур.")


def main():
    game_type = input("Здравствуйте! Выберите игру (chess/chess_with_wizard/checkers): ").strip().lower()
    while game_type not in ['chess', 'chess_with_wizard', 'checkers']:
        print("Некорректный ввод. Пожалуйста введите 'chess', 'chess_with_wizard' или 'checkers'.")
        game_type = input("Выберите игру: ").strip().lower()

    board = Board(game_type)
    turn = 'white'

    while True:
        board.display()
        print(f"Ход {'белых' if turn == 'white' else 'черных'}")

        command = input("Введите команду (например, 'e2 e4', 'undo', 'hint e2', 'threats'): ").strip().split()

        if not command:
            continue

        if command[0] == 'undo':
            if board.undo_move():
                turn = 'black' if turn == 'white' else 'white'
            continue

        if command[0] == 'hint':
            if len(command) < 2:
                print("Некорректный ввод. Попробуйте снова.")
                continue
            try:
                col = ord(command[1][0].lower()) - ord('a')
                row = 8 - int(command[1][1])
                board.show_hints((row, col))
            except (ValueError, IndexError):
                print("Некорректный ввод. Попробуйте снова.")
            continue

        if command[0] == 'threats':
            board.display_threats(turn)
            continue

        if len(command) < 2:
            print("Некорректный ввод. Попробуйте снова.")
            continue

        try:
            start_col = ord(command[0][0].lower()) - ord('a')
            start_row = 8 - int(command[0][1])
            end_col = ord(command[1][0].lower()) - ord('a')
            end_row = 8 - int(command[1][1])
        except (ValueError, IndexError):
            print("Некорректный ввод. Попробуйте снова.")
            continue

        if board.move_piece((start_row, start_col), (end_row, end_col), turn):
            turn = 'black' if turn == 'white' else 'white'


if __name__ == "__main__":
    main()

Некорректный ввод. Пожалуйста введите 'chess', 'chess_with_wizard' или 'checkers'.
  a b c d e f g h
8 r n b w k b n r 8
7 p p p p p p p p 7
6 . . . . . . . . 6
5 . . . . . . . . 5
4 . . . . . . . . 4
3 . . . . . . . . 3
2 P P P P P P P P 2
1 R N B W K B N R 1
  a b c d e f g h
Ход белых
